In [805]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [806]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [865]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
# baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
baseurl = 'https://blocmarket.herokuapp.com/'
#baseurl = 'http://127.0.0.1:5000/'
#baseurl = 'http://127.0.0.1:7000/'

In [866]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [867]:
response.json()

{'signingKey_hex': 'fc480d91e93ef8d1953fe724ea166f32a16f70a349409503a7dcac2fae72686d',
 'traderId': '1',
 'verifyKey_hex': '477e9a0ab1402d9670e2fa2101a40df6937aea8e718ac6f3ecab06eb26c6d284'}

In [841]:
url = baseurl + 'createMarket'

mkRtId = 6
mkId=1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [842]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkRtId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [843]:
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,6,2


In [844]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk,
                     "traderId": tId,
                     "verifyKey_hex": vk,
                     "marketId": mkId,
                     "price": 0.55,
                     "quantity":1}

In [845]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [846]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': 1,
 'traderId': '2'}

In [847]:
# View json response
pd.DataFrame(response.json(), index=[0])


,checks,marketId,price,quantity,traderId
0,True,1,0.55,1,2


In [848]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [849]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': -1,
 'traderId': '2'}

In [850]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.360245943069458 seconds.
matched trade: 1 in 0.367264986038208 seconds.
matched trade: 2 in 0.34751296043395996 seconds.
matched trade: 3 in 0.33552002906799316 seconds.
matched trade: 4 in 0.3351478576660156 seconds.


In [851]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 0.28237295150756836 seconds.
market created: 2 in 0.453995943069458 seconds.
market created: 3 in 0.6857180595397949 seconds.
market created: 4 in 1.3051021099090576 seconds.


In [852]:
tId

'2'

In [853]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 2}

In [854]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [855]:
a = pd.read_json(response.json())
a

,marketId,marketRootId,marketBranchId,marketMin,marketMax
0,1,6,1,0,1.0
1,2,1,1,0,0.7
2,3,2,1,0,0.7
3,4,3,1,0,0.7
4,5,4,1,0,0.7


In [856]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [857]:
a = pd.read_json(response.json())
a

,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,1,0.55,1,1,True,2019-01-20 06:12:20.697
1,1,0.55,-1,1,True,2019-01-20 06:13:05.678
10,1,0.55,-1,2,True,2019-01-20 07:35:27.630
11,1,0.55,1,2,True,2019-01-20 07:35:27.798
12,1,0.55,-1,2,True,2019-01-20 07:35:27.966
13,1,0.55,1,2,True,2019-01-20 07:35:28.135
2,1,0.55,1,2,True,2019-01-20 07:35:23.397
3,1,0.55,-1,2,True,2019-01-20 07:35:24.956
4,1,0.55,-1,2,True,2019-01-20 07:35:26.579
5,1,0.55,1,2,True,2019-01-20 07:35:26.749


In [858]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    content_maketrade['price'] = 0.43
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 1.351715087890625 seconds.
unmatched trade: 1 in 1.4378807544708252 seconds.
unmatched trade: 2 in 1.3318538665771484 seconds.
unmatched trade: 3 in 1.2808799743652344 seconds.
unmatched trade: 4 in 1.3410429954528809 seconds.


In [859]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.43,
 'quantity': -1,
 'traderId': '2'}

In [860]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [861]:
pd.read_json(response.json())

,marketId,price,quantity,traderId,timeStampUTC
14,1,0.43,-1,2,2019-01-20 07:35:41.448
15,1,0.43,-1,2,2019-01-20 07:35:42.884
16,1,0.43,-1,2,2019-01-20 07:35:44.218
17,1,0.43,-1,2,2019-01-20 07:35:45.499
18,1,0.43,-1,2,2019-01-20 07:35:46.839


In [862]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [863]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,0.55,0,1
1,1,0.55,0,2


In [864]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 2.5399410724639893 seconds.
matched trade: 1 in 2.658452033996582 seconds.
matched trade: 2 in 2.728566884994507 seconds.
matched trade: 3 in 2.5069499015808105 seconds.
matched trade: 4 in 2.759593963623047 seconds.



# Curl 



## Create user

~~~~
curl --header "Content-Type: application/json" -X POST http://127.0.0.1:7000/createUser
~~~~
        
## Create market

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketRootId": 1, 
"marketBranchId": 5, 
"marketMin": 0, 
"marketMax": 1}' 
--header "Content-Type: application/json" -X POST http://localhost:7000/createMarket
~~~~
    
## Create trade

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketId": 1,
"price": 0.1212,
"quantity": 1}' --header "Content-Type: application/json" -X POST http://localhost:7000/createTrade

~~~~
        
## View order book (output to file)

~~~~
curl --header "Content-Type: application/json" -X POST http://127.0.0.1:7000/viewOrderBook --output ob.txt
~~~~    
        
Also can load json from file with something like 
~~~~ 
curl -d "@data.json" -X POST http://localhost:3000/data 
~~~~

Matlab will (probably?) run these with 
~~~~
[A, curlout] = system('curlstring')
~~~~

    

In [740]:
str(content_makemarket)


"{'signingKey_hex': '29ba39f7fea00b2639839d7740e875e884cdb593da4b46af8d4d9af3cc212bf0', 'traderId': 1, 'verifyKey_hex': 'e6376aef9f29b5ba13d98f388a2463d42e58300a4438545e769dbfa2a7432f07', 'marketRootId': 4, 'marketBranchId': 1, 'marketMin': 0, 'marketMax': 0.7}"

In [747]:
str(datetime.datetime.utcnow())

'2019-01-20 01:01:08.007049'

# Setup for release

- Get extra database rows on Heroku
- Single 